# TITANIC PREDICTIONS

In [ ]:
#Imports
import pandas as pd
import numpy as np

#Imports for Machine Learning Model
from sklearn.linear_model import LogisticRegression

#Import the Metrics
from sklearn.metrics import classification_report,confusion_matrix

#Import Train_test_split
from sklearn.model_selection import train_test_split

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Reading the Modified Titanic Data from Titanic Visualisations

The Following Code has been taken from to import data from another ipynb notebook
**http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Importing%20Notebooks.ipynb** 

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]    
    
sys.meta_path.append(NotebookFinder())

#### Importing the Data

In [89]:
from Titanic_Visualisations import titanic_data

In [101]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Ages categorized
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Young Adults
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Young Adults
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Young Adults
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Young Adults
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Young Adults


### Converting Categorical Features 

In [99]:
cat_features = ['Embarked','Sex','Pclass']

In [102]:
# drop_first is set to be true , because out of 3 values, when 2 values are filled, the third can be 
# inferred from the two itself

titanic_data = pd.get_dummies(titanic_data,columns=cat_features,drop_first=True)

#### Train Test Split to get the test data and train data

In [117]:
X = titanic_data.drop(['Survived','Name','Ticket','Ages categorized'],axis = 1)
y = titanic_data['Survived']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

#### Instance of the Logistic Regression

In [118]:
logreg = LogisticRegression()

In [119]:
# Fit the Train Data to the model
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [121]:
#Predictions
predictions = logreg.predict(X_test)

### Evaluation of the Prediction model using Logistic Regression

In [122]:
print(confusion_matrix(y_test,predictions))

[[147  20]
 [ 31  69]]


In [123]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.83      0.88      0.85       167
          1       0.78      0.69      0.73       100

avg / total       0.81      0.81      0.81       267

